In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [2]:
path = "D:\\Anadolu_Hayat_Datathon\\"
train_df=pd.read_csv(path+"train.csv")
train_df

<ipython-input-2-d08dcff3c983>:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df=pd.read_csv(path+"train.csv")


,MUSTERI_ID,LABEL,FLAG,PP_CINSIYET,PP_YAS,PP_MESLEK,PP_MUSTERI_SEGMENTI,PP_UYRUK,IL,SORU_YATIRIM_KARAKTERI_CVP,...,HU11,HU12,HU13,HU14,HU15,HU16,HU17,HU18,HU19,AKTIF_ILK_POLICE_RG
0,61921038d5c39a821cfed2e5e3ad88740decd1fbd6f34b...,UA,HAZIRAN,2,505,Öğretmen,105,1,34,NaN,...,0,0,0,0,0,0,0,0,0,9
1,8cd400d3897e5ff130e8e942ee1906d17531a8253d26a4...,UA,HAZIRAN,2,889,Emekli (Sosyal Güvenlik Kurumu Bilinmiyor),102,1,20,NaN,...,0,0,0,0,0,0,0,0,0,11
2,6af95ba9ebe2362a0b31b4443fb2405b11d588780c01ad...,HU19,HAZIRAN,2,578,Muhasebe,104,1,6,Yetkin,...,0,1,0,0,0,0,0,0,0,66
3,fe33e7873368d603d2a49a982619e8d61a964d7a0ef68e...,UA,MAYIS,2,648,Diğer,106,1,34,NaN,...,0,0,0,0,0,0,0,0,0,111
4,aae8411e2c335e94922290314abe1fcc81044e8071e1c8...,UA,EYLUL,2,487,İşçi,103,1,77,NaN,...,0,0,0,0,0,0,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852714,4a245f1a49af39319faa41668d6ee425fb71c7a75e8b22...,UA,MAYIS,1,551,İşçi,105,1,35,NaN,...,0,0,0,0,0,0,0,0,0,7
852715,2087a8e5f1f98d143011df1cccaa0331e3427b1a1e642e...,UA,HAZIRAN,2,466,Ev Hanımı,104,1,33,NaN,...,0,0,0,0,0,0,0,0,0,52
852716,8d4dccb1bd4763e909476935805d99b84e7f2098b646b7...,UA,MAYIS,1,139,İşçi,106,1,28,NaN,...,0,0,0,0,0,0,0,0,0,5
852717,e33a48c42db1da438d54da64cf59ef4acae4583bd4effc...,UA,MAYIS,2,303,NaN,106,1,20,NaN,...,0,0,0,0,0,0,0,0,0,72


In [3]:
turn_to_nation = {1: 'TC Vatandaşı', 2: 'Mavi Kart',3:'Yabancı Uyruklu'}

# Dönüşümün gerçekleştirilmesi
train_df['PP_UYRUK'] = train_df['PP_UYRUK'].replace(turn_to_nation)

In [4]:
train_df['SORU_GELIR_CVP'] = train_df['SORU_GELIR_CVP'].str.replace(',', '.').astype(float)

In [5]:
for col in train_df.columns:
    print(col)
    print(train_df[col].unique())

MUSTERI_ID
['61921038d5c39a821cfed2e5e3ad88740decd1fbd6f34ba975819dd9907482e7'
 '8cd400d3897e5ff130e8e942ee1906d17531a8253d26a466b2ad97023bfd5bfd'
 '6af95ba9ebe2362a0b31b4443fb2405b11d588780c01ad744bdb43e9419b875f' ...
 '8d4dccb1bd4763e909476935805d99b84e7f2098b646b7f1f7cd3f41ed5d60fb'
 'e33a48c42db1da438d54da64cf59ef4acae4583bd4effcb727648c91269d7a16'
 '59fd829fc435bf3ed0aa832ca0c89397e5197d8cc135187362d77495cc5d5bdd']
LABEL
['UA' 'HU19' 'HU07' 'HU14' 'HU11' 'HU15' 'HU12' 'HU06']
FLAG
['HAZIRAN' 'MAYIS' 'EYLUL' 'EKIM' 'TEMMUZ' 'AGUSTOS']
PP_CINSIYET
[2 1]
PP_YAS
[ 505  889  578 ... 1136 1133 1174]
PP_MESLEK
['Öğretmen' 'Emekli (Sosyal Güvenlik Kurumu Bilinmiyor)' 'Muhasebe'
 'Diğer' 'İşçi' 'Serbest Meslek' 'Öğrenci' nan 'İşyeri Sahibi/Ortağı'
 'Esnaf' 'Satış' 'Ev Hanımı' 'Mimar'
 'SSK-Sosyal Sigortalar Kurumu Emeklileri' 'Akademisyen'
 'Yazılımcı/Programcı/Analist' 'Banka' 'Danışman' 'Hizmet Çalışanı'
 'Sigorta' 'Memur' 'Test Uzmanı' 'Çiftçi' 'Eczacı' 'Polis' 'Terzi'
 'Tacir ve Esnaf'

In [6]:
nan_counts = train_df.isna().sum()
nan_counts[nan_counts > 0]

PP_MESLEK                      88620
SORU_YATIRIM_KARAKTERI_CVP    720021
SORU_YATIRIM_KARAKTERI_RG     720021
SORU_MEDENI_HAL_CVP             1780
SORU_MEDENI_HAL_RG              1780
SORU_EGITIM_CVP               127095
SORU_EGITIM_RG                127095
SORU_GELIR_CVP                422491
SORU_GELIR_RG                 277902
SORU_COCUK_SAYISI_CVP         416560
SORU_COCUK_SAYISI_RG          416560
SON_AY_KATKI_MIKTARI          851739
SON_AY_KATKI_ADET             851739
SON_CEYREK_KATKI_MIKTARI      847099
SON_CEYREK_KATKI_ADET         847099
SON_SENE_KATKI_MIKTARI        837713
SON_SENE_KATKI_ADET           837713
dtype: int64

In [7]:
train_df= train_df.drop(columns=["SON_AY_KATKI_MIKTARI","SON_AY_KATKI_ADET",
                               "SON_CEYREK_KATKI_MIKTARI","SON_CEYREK_KATKI_ADET",
                               "SON_SENE_KATKI_MIKTARI","SON_SENE_KATKI_ADET",
                                "SORU_YATIRIM_KARAKTERI_CVP","SORU_YATIRIM_KARAKTERI_RG"])

### Yeni Sütun Oluşturma

#### Bölge Sütunu Oluşturma ( Plaka kodlarına göre)

In [8]:
regions = {
    "Marmara ": ["10", "11", "16", "17", "22", "34", "39", "41", "54", "59", "77"],
    "Ege ": ["3", "9", "20", "35", "43", "45", "48", "64"],
    "Akdeniz ": ["1", "7", "15", "31", "32", "33", "46", "80"],
    "IA ": ["6", "18", "26", "38", "40", "42", "50", "51", "58", "66", "68", "70", "71"],
    "Karadeniz ": ["5", "8", "14", "19", "28", "29", "37", "52", "53", "55", "57", "60", "61", "67", "69", "74", "78", "81"],
    "DA ": ["4", "12", "13", "23", "24", "25", "30", "36", "44", "49", "62", "65", "75", "76"],
    "GA ": ["2", "21", "27", "47", "56", "63", "72", "73", "79"]
};

def find_region(il):
    for region, plates in regions.items():
        if(np.isnan(il)):
            return None
        elif str(int(il)) in plates:
            return region
    return None  # Eğer plaka hiçbir bölge ile eşleşmezse

train_df['BOLGE'] = train_df['IL'].apply(find_region)

In [9]:
train_df["BOLGE"].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
852714    False
852715    False
852716    False
852717    False
852718    False
Name: BOLGE, Length: 852719, dtype: bool

#### Yaş Grubu Sütunu Oluşturma

In [10]:
age_groups = {
    "18-25": list(range(18, 26)),
    "26-35": list(range(26, 36)),
    "36-45": list(range(36, 46)),
    "46-55": list(range(46, 56)),
    "56-65": list(range(56, 66)),
    "65+": list(range(66, 120))  # Burada 120, yaş grubunun sonunu temsil eden bir tahmini maksimum yaş
}

def find_ageGroup(yas):
    for group, ages in age_groups.items():
        if (yas//12) <= max(ages):
            return group
    return None  # Eğer plaka hiçbir bölge ile eşleşmezse

train_df['YAS_GRUBU'] = train_df['PP_YAS'].apply(find_ageGroup)

#### Vade Tutarı İle İlgili İşlemler

In [11]:
def findVadeOrtalama(dfs):
    vade_sutunlari = [f'VADE_TUTAR_{i}' for i in range(12)]

    # Sadece vade tutarlarını içeren sütunları al
    vade_tutarlari = dfs[vade_sutunlari]

    # Satır bazında ortalamayı al ve yeni bir sütun oluştur
    dfs['VADE_ORTALAMASI'] = vade_tutarlari.mean(axis=1)
findVadeOrtalama(train_df)#### Ortalama Vade Değişim Hızı

#### Ortalama Vade Değişim Hızı

In [12]:
def findVadeDegisim(dfs):
    vade_sutunlari = [f'VADE_TUTAR_{i}' for i in range(12)]
    vade_degisimleri = dfs[vade_sutunlari].diff(axis=1)  # Önceki ay ile karşılaştırarak değişim miktarlarını hesapla

    # Ortalama vade tutarı değişim hızını hesapla
    dfs['ORTALAMA_VADE_TUTARI_DEGISIM_HIZI'] = vade_degisimleri.mean(axis=1)
findVadeDegisim(train_df)#### Ortalama Vade Değişim Hızı#### ÖdemeTutarı ile ilgili İşlemler

#### ÖdemeTutarı ile ilgili İşlemler

In [13]:
def findOdemeToplamı(dfs):
    odeme_sutunlari = [f'ODEME_TUTAR_{i}' for i in range(12)]
    odeme_tutarlari = dfs[odeme_sutunlari]
    dfs['ODEME TOPLAMI'] = odeme_tutarlari.sum(axis=1)   
findOdemeToplamı(train_df)

#### Toplam Talep Sayısı Sütunu Oluşturma

In [14]:
train_df["TOPLAM TALEP"] = train_df["BES_AYRILMA_TALEP_ADET"]+train_df["HAYAT_AYRILMA_TALEP_ADET"]+train_df["BILGI_TALEP_ADET"]+train_df["ODEMEME_TALEP_ADET"]                  

In [15]:
train_df['Talep_Sayısı_Kategorisi'] = pd.cut(train_df['TOPLAM TALEP'], bins=[-1, 0, 1, 3, float('inf')], labels=['0', '1', '2-3', '4+'])

#### Ekonomik Durum Sütunu Oluşturma

#### Meslek Grubu Oluşturma

In [16]:
# Savcı, hakim, ve Hakim/Savcı diye bir şey var.
# jobs = {
#     "Çalışmıyor":["Öğrenci","Ev Hanımı","Emekli (Sosyal Güvenlik Kurumu Bilinmiyor)","SSK-Sosyal Sigortalar Kurumu Emeklileri",
#                   "Bağkur Emeklileri","Emekli Sandığı Emeklileri","Emekli","İşsiz","Emekli (Vakıf,Sandık v.b.)","Çalışmıyor",
#                   "Üniversite Öğrencisi"],
#     "Emekli": ["Emekli (Sosyal Güvenlik Kurumu Bilinmiyor)","SSK-Sosyal Sigortalar Kurumu Emeklileri",
#                   "Bağkur Emeklileri","Emekli Sandığı Emeklileri","Emekli","Emekli (Vakıf,Sandık v.b.)"],
    
#     "Teknoloji":['Yazılımcı/Programcı/Analist', 'Mühendis','Teknisyen/Tekniker','Yazılımcı','Yazılım Geliştirme Uzmanı','Teknisyen'],
#     "Sağlık":['Doktor','Eczacı',"Sağlık Personeli","Hekim","Hemşire","Veteriner Hekim","Laborant","Biyolog","Ebe","Diş Hekimi"],
    
#     "Kamu":['Doktor',"Öğretmen","Polis","Memur","Akademisyen",'Kamu Çalışanı',"Hakim/Savcı","Ordu Mensubu","Subay","Astsubay",
#             "Büro Elemanı","Ofis Elemanı","Denetçi","Müfettiş/Denetçi","Emniyet Mensubu","Uzman Erbaş","Hakim","Noter","Din Görevlisi",
#             "Bürokrat","General","Uzman Jandarma","Savcı","Amiral"],
#     "Askeri":["Ordu Mensubu","Polis","Subay","Astsubay","Emniyet Mensubu","Uzman Erbaş","General","Uzman Jandarma","Amiral"],
    
#     "Özel":['Yazılımcı/Programcı/Analist', 'Mühendis','Teknisyen/Tekniker','Yazılımcı','Yazılım Geliştirme Uzmanı','Teknisyen',
#             'Muhasebe', 'İşçi', 'Serbest Meslek', 'İşyeri Sahibi/Ortağı','Esnaf', 'Satış', 'Mimar','Banka',
#             'Danışman', 'Hizmet Çalışanı','Sigorta', 'Test Uzmanı', 'Çiftçi',  'Terzi','Tacir ve Esnaf','Finans Sektörü Çalışan',
#             'İş Adamı', 'Şoför', 'Satış/Pazarlama Çalışanı','Avukat', 'Tüccar','İletişim','Pazarlama', 'Profesyonel Meslek Mensupları',
#             'Sanatçı', 'Mali Müşavir/Muhasebeci','Halkla İlişkiler', 'Medya', 'İş Analisti', 'Finans', 'Analist','Güvenlik Görevlisi',
#             'Mali Müşavir','Aşçı','Yazar', 'CEO'  'Müteahhit', 'Reklamcı/Halka İlişkiler','Sporcu', 
#             'Şoför/Makine Operatörü', 'Medya Mensubu', 'Garson', 'Üst Düzey Yönetici', 'Sivil Pilot', 'Turizmci','Radyolog', 
#             'Departman Yöneticisi', 'Reklam', 'Bakıcı', 'Komi','Zanaatkar', 'Kimyager', 'Pilot/Kaptan','Antrenör','Takım/Ekip Lideri',
#             'Emlakçı', 'Yönetim Kurulu Üyesi', 'Proje Yöneticisi', 'Barmen', 'Kuyumcu/Döviz Bürosu Çalışanı','Şair', 'Balıkçı'],
    
#     "Ekonomi":["Muhasebe","Sigorta","Banka","Finans Sektörü Çalışan","Mali Müşavir/Muhasebeci","Finans","Mali Müşavir","Kuyumcu/Döviz Bürosu Çalışanı"],
#     "Diğer":["Diğer","Bilinmiyor","nan"]
# }
# def find_jobGroup(job):
#     for group, jobss in jobs.items():
#         if job in jobss:
#             return group
#     return None  # Eğer meslek hiçbir bölge ile eşleşmezse
# train_df['SEKTOR'] = train_df['PP_MESLEK'].apply(find_jobGroup)
# train_df['SEKTOR']

### Non-Numeric Featurelar

* Label
* FLAG
* PP_Meslek
* SORU_YATIRIM_KARAKTERI_CVP
* SORU_MEDENI_HAL_CVP
* SORU_EGITIM_CVP

## Kategorik Sütun Encodingi

In [17]:
train_df['FLAG'] = train_df['FLAG'].replace('MAYIS', 5)
train_df['FLAG'] = train_df['FLAG'].replace('HAZIRAN', 6)
train_df['FLAG'] = train_df['FLAG'].replace('TEMMUZ', 7)
train_df['FLAG'] = train_df['FLAG'].replace('AGUSTOS', 8)
train_df['FLAG'] = train_df['FLAG'].replace('EYLUL', 9)
train_df['FLAG'] = train_df['FLAG'].replace('EKIM', 10)
train_df

,MUSTERI_ID,LABEL,FLAG,PP_CINSIYET,PP_YAS,PP_MESLEK,PP_MUSTERI_SEGMENTI,PP_UYRUK,IL,SORU_MEDENI_HAL_CVP,...,HU18,HU19,AKTIF_ILK_POLICE_RG,BOLGE,YAS_GRUBU,VADE_ORTALAMASI,ORTALAMA_VADE_TUTARI_DEGISIM_HIZI,ODEME TOPLAMI,TOPLAM TALEP,Talep_Sayısı_Kategorisi
0,61921038d5c39a821cfed2e5e3ad88740decd1fbd6f34b...,UA,6,2,505,Öğretmen,105,TC Vatandaşı,34,Single,...,0,0,9,Marmara,36-45,260.416667,-36.818182,3125,0,0
1,8cd400d3897e5ff130e8e942ee1906d17531a8253d26a4...,UA,6,2,889,Emekli (Sosyal Güvenlik Kurumu Bilinmiyor),102,TC Vatandaşı,20,Married,...,0,0,11,Ege,65+,3340.333333,-520.000000,41998,0,0
2,6af95ba9ebe2362a0b31b4443fb2405b11d588780c01ad...,HU19,6,2,578,Muhasebe,104,TC Vatandaşı,6,Single,...,0,0,66,IA,46-55,856.416667,-49.181818,10277,0,0
3,fe33e7873368d603d2a49a982619e8d61a964d7a0ef68e...,UA,5,2,648,Diğer,106,TC Vatandaşı,34,Married,...,0,0,111,Marmara,46-55,395.333333,-24.363636,0,0,0
4,aae8411e2c335e94922290314abe1fcc81044e8071e1c8...,UA,9,2,487,İşçi,103,TC Vatandaşı,77,Married,...,0,0,90,Marmara,36-45,388.666667,-19.181818,1377,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852714,4a245f1a49af39319faa41668d6ee425fb71c7a75e8b22...,UA,5,1,551,İşçi,105,TC Vatandaşı,35,Married,...,0,0,7,Ege,36-45,905.000000,-39.090909,10860,0,0
852715,2087a8e5f1f98d143011df1cccaa0331e3427b1a1e642e...,UA,6,2,466,Ev Hanımı,104,TC Vatandaşı,33,Married,...,0,0,52,Akdeniz,36-45,999.333333,-57.454545,11992,0,0
852716,8d4dccb1bd4763e909476935805d99b84e7f2098b646b7...,UA,5,1,139,İşçi,106,TC Vatandaşı,28,Single,...,0,0,5,Karadeniz,18-25,221.666667,-13.000000,0,0,0
852717,e33a48c42db1da438d54da64cf59ef4acae4583bd4effc...,UA,5,2,303,NaN,106,TC Vatandaşı,20,Single,...,0,0,72,Ege,18-25,296.000000,-18.272727,0,0,0


### Dış Veri Ekleme

#### Enflasyon, Döviz

In [18]:
# Verileri içeren metin tablosunu oluştur
#11 61.98 3.28
data = """
10 61.36 3.43
09 61.53 4.75
08 58.94 9.09
07 47.83 9.49
06 38.21 3.92
05 39.59 0.04
"""
# StringIO kullanarak metin tablosunu oku ve DataFrame oluştur
df = pd.read_csv(StringIO(data), sep='\s+', header=None, names=['FLAG', 'Inflation', 'Value2'])
df = df.drop("Value2", axis = 1)
df = df.sort_values(by='FLAG')
df =df.reset_index()
df = df.drop("index", axis = 1)
display(df)


# 6	11	28.637082	30.902518
# 7	12	29.074857	31.714886
df2 = pd.read_excel(path+"veri.xlsx")
df2 = df2.drop(df2.index[:40])
df2 = df2.drop(df2.index[6:])
df2 = df2.drop(columns=["TP DK GBP S YTL","TP FG J0","Tarih"],axis = 1)
df2 =df2.reset_index()
df2 = df2.drop("index", axis = 1)
df2["FLAG"] = 5 + df2.index 
display(df2)

forecast_df = pd.DataFrame(index=[6, 7])
for i in ["TP DK USD S YTL","TP DK EUR S YTL",]:
    data = df2[[i,"FLAG"]]
    model = ARIMA(data[i], order=(1, 2, 1))  

    # Modeli eğitme
    fit_model = model.fit()

    # Tahmin yapma
    forecast = fit_model.forecast(steps=2)  # Gelecek 2 adım için tahmin yapılır

    forecast_df[i] = forecast
#     print("ARIMA Forecast:", forecast)

model = ARIMA(df["Inflation"], order=(1, 2, 1))  

# Modeli eğitme
fit_model = model.fit()

# Tahmin yapma
forecast = fit_model.forecast(steps=2)  
forecast_df["Inflation"] = forecast
print("ARIMA Forecast:", forecast)
display(forecast)

# Sütunu DataFrame'in başına ekleme
forecast_df.insert(0, 'FLAG', ['11','12'])
df2['Inflation'] = df['Inflation'].iloc[:]
df2 = pd.concat([df2, forecast_df])
df2

,FLAG,Inflation
0,5,39.59
1,6,38.21
2,7,47.83
3,8,58.94
4,9,61.53
5,10,61.36


,TP DK USD S YTL,TP DK EUR S YTL,FLAG
0,19.719514,21.464086,5
1,23.106042,24.989216,6
2,26.470338,29.256700,7
3,26.999127,29.481700,8
4,26.989448,28.872410,9
5,27.846114,29.409641,10


C:\Users\ACER\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


ARIMA Forecast: 6    62.352200
7    63.281504
Name: predicted_mean, dtype: float64


C:\Users\ACER\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


6    62.352200
7    63.281504
Name: predicted_mean, dtype: float64

,TP DK USD S YTL,TP DK EUR S YTL,FLAG,Inflation
0,19.719514,21.464086,5,39.590000
1,23.106042,24.989216,6,38.210000
2,26.470338,29.256700,7,47.830000
3,26.999127,29.481700,8,58.940000
4,26.989448,28.872410,9,61.530000
5,27.846114,29.409641,10,61.360000
6,28.563131,30.289812,11,62.352200
7,29.354117,31.421244,12,63.281504


In [19]:
train_df = pd.merge(train_df,df2,on='FLAG', how='left' )

### NaN Handling

In [20]:
nan_counts = train_df.isna().sum()
nan_counts[nan_counts > 0]

PP_MESLEK                 88620
SORU_MEDENI_HAL_CVP        1780
SORU_MEDENI_HAL_RG         1780
SORU_EGITIM_CVP          127095
SORU_EGITIM_RG           127095
SORU_GELIR_CVP           422491
SORU_GELIR_RG            277902
SORU_COCUK_SAYISI_CVP    416560
SORU_COCUK_SAYISI_RG     416560
BOLGE                      3222
YAS_GRUBU                     1
dtype: int64

In [21]:
column_names_with_nan = nan_counts[nan_counts > 0].index.tolist()
for col in column_names_with_nan:
    print(col)
    if(col not in ["BOLGE","YAS_GRUBU","PP_MESLEK","Talep_Sayısı_Kategorisi","SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP"]):
        train_df[col].fillna(-train_df[col].median(), inplace=True)

PP_MESLEK
SORU_MEDENI_HAL_CVP
SORU_MEDENI_HAL_RG
SORU_EGITIM_CVP
SORU_EGITIM_RG
SORU_GELIR_CVP
SORU_GELIR_RG
SORU_COCUK_SAYISI_CVP
SORU_COCUK_SAYISI_RG
BOLGE
YAS_GRUBU


In [22]:
bins = [-101, 1000,10000,50000,100000,500000,1000000,10000000,1000000000000]

train_df['SORU_GELIR_CVP'] = pd.cut(train_df['SORU_GELIR_CVP'], bins)

train_df['SORU_GELIR_CVP'].value_counts()

SORU_GELIR_CVP
(1000, 10000]                255749
(-101, 1000]                 124571
(10000, 50000]                45029
(50000, 100000]                3219
(100000, 500000]               1416
(1000000, 10000000]              94
(500000, 1000000]                93
(10000000, 1000000000000]        57
Name: count, dtype: int64

In [23]:
cols = ["SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP","PP_MESLEK"
       ,"YAS_GRUBU","BOLGE","PP_UYRUK","SORU_GELIR_CVP","Talep_Sayısı_Kategorisi"]
for col in cols  :
    one_hot_encoded = pd.get_dummies(train_df[col])
    one_hot_encoded = one_hot_encoded.astype(int)
    train_df = pd.concat([train_df, one_hot_encoded], axis=1)
    train_df = train_df.drop(col, axis=1,)

In [24]:
rg_cols = ["SORU_EGITIM_RG","SORU_GELIR_RG","SORU_COCUK_SAYISI_RG","SORU_MEDENI_HAL_RG","SORU_EGITIM_RG"]
for col in rg_cols:
    train_df[col]+= 11-train_df["FLAG"]
train_df = train_df.drop("FLAG", axis =1)

In [25]:
train_df.columns = train_df.columns.astype(str)

In [26]:
train_df.columns.tolist()

['MUSTERI_ID',
 'LABEL',
 'PP_CINSIYET',
 'PP_YAS',
 'PP_MUSTERI_SEGMENTI',
 'IL',
 'SORU_MEDENI_HAL_RG',
 'SORU_EGITIM_RG',
 'SORU_GELIR_RG',
 'SORU_COCUK_SAYISI_CVP',
 'SORU_COCUK_SAYISI_RG',
 'BES_AYRILMA_TALEP_ADET',
 'ODEMEME_TALEP_ADET',
 'HAYAT_AYRILMA_TALEP_ADET',
 'BILGI_TALEP_ADET',
 'VADE_TUTAR_0',
 'ODEME_TUTAR_0',
 'VADE_TUTAR_1',
 'ODEME_TUTAR_1',
 'VADE_TUTAR_2',
 'ODEME_TUTAR_2',
 'VADE_TUTAR_3',
 'ODEME_TUTAR_3',
 'VADE_TUTAR_4',
 'ODEME_TUTAR_4',
 'VADE_TUTAR_5',
 'ODEME_TUTAR_5',
 'VADE_TUTAR_6',
 'ODEME_TUTAR_6',
 'VADE_TUTAR_7',
 'ODEME_TUTAR_7',
 'VADE_TUTAR_8',
 'ODEME_TUTAR_8',
 'VADE_TUTAR_9',
 'ODEME_TUTAR_9',
 'VADE_TUTAR_10',
 'ODEME_TUTAR_10',
 'VADE_TUTAR_11',
 'ODEME_TUTAR_11',
 'ANAPARA',
 'GETIRI',
 'BU01',
 'BU02',
 'BU03',
 'BU04',
 'BU05',
 'BU06',
 'BU07',
 'BU08',
 'BU09',
 'BU10',
 'BU11',
 'BU12',
 'BU13',
 'BU14',
 'BU15',
 'BU16',
 'BU17',
 'BU18',
 'BU19',
 'BU20',
 'BU21',
 'BU22',
 'BU23',
 'BU24',
 'HU01',
 'HU02',
 'HU03',
 'HU04',
 'HU05'

### Test Set

In [27]:
test_df = pd.read_csv(path+"test.csv")
test_df.head()

<ipython-input-27-afbbdc826f1e>:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(path+"test.csv")


,MUSTERI_ID,FLAG,PP_CINSIYET,PP_YAS,PP_MESLEK,PP_MUSTERI_SEGMENTI,PP_UYRUK,IL,SORU_YATIRIM_KARAKTERI_CVP,SORU_YATIRIM_KARAKTERI_RG,...,HU11,HU12,HU13,HU14,HU15,HU16,HU17,HU18,HU19,AKTIF_ILK_POLICE_RG
0,15b29e51dff893ac19bcbc96c96d1cda0307b4e0edafcd...,KASIM,2,467,Öğretmen,103,1,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,100
1,7532c2d4751858dd8e7eb3f26c61af568fd3b2a6aa2f32...,KASIM,1,658,Öğretmen,106,1,54.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,111
2,47498c1c6161aea484b0eed7997846919ca9287684aee4...,KASIM,1,634,Serbest Meslek,106,1,48.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,127
3,48fe7f96c7e04326f1669b4ae5bdc5ae030da6b8c26498...,KASIM,2,638,NaN,106,1,67.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,178
4,26dbe514ebfdb7b664760c07f94d0bf0b297528cd5a358...,KASIM,1,80,Öğrenci,106,1,34.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,3


In [28]:
test_df['PP_UYRUK'] = test_df['PP_UYRUK'].replace(turn_to_nation)

In [29]:
test_df['SORU_GELIR_CVP'] = test_df['SORU_GELIR_CVP'].str.replace(',', '.').astype(float)

In [30]:
nan_counts = test_df.isna().sum()
nan_counts[nan_counts > 0]

PP_MESLEK                      26439
IL                                 1
SORU_YATIRIM_KARAKTERI_CVP    168244
SORU_YATIRIM_KARAKTERI_RG     168244
SORU_MEDENI_HAL_CVP              316
SORU_MEDENI_HAL_RG               316
SORU_EGITIM_CVP                27084
SORU_EGITIM_RG                 27084
SORU_GELIR_CVP                113699
SORU_GELIR_RG                  79699
SORU_COCUK_SAYISI_CVP         113535
SORU_COCUK_SAYISI_RG          113535
SON_AY_KATKI_MIKTARI          195303
SON_AY_KATKI_ADET             195303
SON_CEYREK_KATKI_MIKTARI      193712
SON_CEYREK_KATKI_ADET         193712
SON_SENE_KATKI_MIKTARI        191885
SON_SENE_KATKI_ADET           191885
dtype: int64

In [31]:
test_df['BOLGE'] = test_df['IL'].apply(find_region)
test_df['YAS_GRUBU'] = test_df['PP_YAS'].apply(find_ageGroup)
findVadeOrtalama(test_df)#### Ortalama Vade Değişim Hızı
findVadeDegisim(test_df)    
findOdemeToplamı(test_df)
test_df["TOPLAM TALEP"] = test_df["BES_AYRILMA_TALEP_ADET"]+test_df["HAYAT_AYRILMA_TALEP_ADET"]+test_df["BILGI_TALEP_ADET"]+test_df["ODEMEME_TALEP_ADET"]                  
test_df['Talep_Sayısı_Kategorisi'] = pd.cut(test_df['TOPLAM TALEP'], bins=[-1, 0, 1, 3, float('inf')], labels=['0', '1', '2-3', '4+'])

In [32]:
test_df['FLAG'] = test_df['FLAG'].replace('KASIM', 11)
test_df['FLAG'] = test_df['FLAG'].replace('ARALIK', 12)

In [33]:
df3 = df2[6:]
df3['FLAG'] = df3['FLAG'].astype('int64')
df3

<ipython-input-33-59c97684d8db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['FLAG'] = df3['FLAG'].astype('int64')


,TP DK USD S YTL,TP DK EUR S YTL,FLAG,Inflation
6,28.563131,30.289812,11,62.352200
7,29.354117,31.421244,12,63.281504


In [34]:
test_df = pd.merge(test_df,df3, on='FLAG', how='left' )
test_df

,MUSTERI_ID,FLAG,PP_CINSIYET,PP_YAS,PP_MESLEK,PP_MUSTERI_SEGMENTI,PP_UYRUK,IL,SORU_YATIRIM_KARAKTERI_CVP,SORU_YATIRIM_KARAKTERI_RG,...,BOLGE,YAS_GRUBU,VADE_ORTALAMASI,ORTALAMA_VADE_TUTARI_DEGISIM_HIZI,ODEME TOPLAMI,TOPLAM TALEP,Talep_Sayısı_Kategorisi,TP DK USD S YTL,TP DK EUR S YTL,Inflation
0,15b29e51dff893ac19bcbc96c96d1cda0307b4e0edafcd...,11,2,467,Öğretmen,103,TC Vatandaşı,1.0,NaN,NaN,...,Akdeniz,36-45,555.833333,-25.181818,0,0,0,28.563131,30.289812,62.352200
1,7532c2d4751858dd8e7eb3f26c61af568fd3b2a6aa2f32...,11,1,658,Öğretmen,106,TC Vatandaşı,54.0,NaN,NaN,...,Marmara,46-55,901.000000,-40.909091,10812,0,0,28.563131,30.289812,62.352200
2,47498c1c6161aea484b0eed7997846919ca9287684aee4...,11,1,634,Serbest Meslek,106,TC Vatandaşı,48.0,NaN,NaN,...,Ege,46-55,2305.666667,-104.363636,0,0,0,28.563131,30.289812,62.352200
3,48fe7f96c7e04326f1669b4ae5bdc5ae030da6b8c26498...,11,2,638,NaN,106,TC Vatandaşı,67.0,NaN,NaN,...,Karadeniz,46-55,626.333333,-28.727273,0,0,0,28.563131,30.289812,62.352200
4,26dbe514ebfdb7b664760c07f94d0bf0b297528cd5a358...,11,1,80,Öğrenci,106,TC Vatandaşı,34.0,NaN,NaN,...,Marmara,18-25,111.333333,-30.363636,1336,0,0,28.563131,30.289812,62.352200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195626,a81089c7050cfa5bdb47d519cfc616f66aa28bc5bda0b5...,12,2,229,NaN,105,TC Vatandaşı,10.0,NaN,NaN,...,Marmara,18-25,83.500000,-30.363636,1002,0,0,29.354117,31.421244,63.281504
195627,fd4362b714e47318627997b801111f34b25c243e6ae550...,11,2,458,Ev Hanımı,105,TC Vatandaşı,22.0,NaN,NaN,...,Marmara,36-45,569.833333,-25.727273,6838,0,0,28.563131,30.289812,62.352200
195628,bf0aba86aa8ece82920ce0fea1d1a013d0f80f3761ad47...,12,1,501,Mühendis,105,TC Vatandaşı,34.0,NaN,NaN,...,Marmara,36-45,79.666667,-43.454545,956,0,0,29.354117,31.421244,63.281504
195629,7cf9130d00d3393bb1a1f3b1815eb267c7546e90fc466a...,12,1,593,Profesyonel Meslek Mensupları,105,TC Vatandaşı,35.0,NaN,NaN,...,Ege,46-55,119.583333,-130.454545,1435,0,0,29.354117,31.421244,63.281504


In [35]:
test_df= test_df.drop(columns=["SON_AY_KATKI_MIKTARI","SON_AY_KATKI_ADET",
                               "SON_CEYREK_KATKI_MIKTARI","SON_CEYREK_KATKI_ADET",
                               "SON_SENE_KATKI_MIKTARI","SON_SENE_KATKI_ADET",
                               "SORU_YATIRIM_KARAKTERI_CVP","SORU_YATIRIM_KARAKTERI_RG","FLAG"])

In [36]:
nan_counts = test_df.isna().sum()
nan_counts[nan_counts > 0]

PP_MESLEK                 26439
IL                            1
SORU_MEDENI_HAL_CVP         316
SORU_MEDENI_HAL_RG          316
SORU_EGITIM_CVP           27084
SORU_EGITIM_RG            27084
SORU_GELIR_CVP           113699
SORU_GELIR_RG             79699
SORU_COCUK_SAYISI_CVP    113535
SORU_COCUK_SAYISI_RG     113535
BOLGE                       623
dtype: int64

In [37]:
column_names_with_nan = nan_counts[nan_counts > 0].index.tolist()
for col in column_names_with_nan:
    if(col not in ["BOLGE","YAS_GRUBU","PP_MESLEK","Talep_Sayısı_Kategorisi","SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP"]):
        test_df[col].fillna(-test_df[col].median(), inplace=True)

In [38]:
test_df = test_df.drop('MUSTERI_ID',axis = 1)
test_df['Marriage Annulled'] = 0

In [39]:
bins = [-101, 1000,10000,50000,100000,500000,1000000,10000000,1000000000000]

test_df['SORU_GELIR_CVP'] = pd.cut(test_df['SORU_GELIR_CVP'], bins)
test_df['SORU_GELIR_CVP'].value_counts()

SORU_GELIR_CVP
(1000, 10000]                44230
(-101, 1000]                 21511
(10000, 50000]               14690
(50000, 100000]               1014
(100000, 500000]               439
(500000, 1000000]               21
(1000000, 10000000]             17
(10000000, 1000000000000]       10
Name: count, dtype: int64

In [40]:
cols = ["SORU_MEDENI_HAL_CVP","SORU_EGITIM_CVP","PP_MESLEK"
       ,"YAS_GRUBU","BOLGE","PP_UYRUK","SORU_GELIR_CVP","Talep_Sayısı_Kategorisi"]
for col in cols  :
    one_hot_encoded = pd.get_dummies(test_df[col])
    one_hot_encoded = one_hot_encoded.astype(int)
    test_df = pd.concat([test_df, one_hot_encoded], axis=1)
    test_df = test_df.drop(col, axis=1,)

In [41]:
test_df.columns = test_df.columns.astype(str)

In [42]:
test_df.isna().sum().tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## Modelling

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE

In [75]:
UA_samples = train_df[(train_df['LABEL'] == 'UA') ].sample(n=130000, random_state=42)

# Diğer sınıflardaki örnekler
other_samples = train_df[train_df['LABEL'] != 'UA']

# Yeni veri setini birleştirelim
balanced_data = pd.concat([UA_samples, other_samples])

X = balanced_data.drop(columns = ['LABEL','MUSTERI_ID'], axis=1)
y = balanced_data['LABEL']

unique_values, counts = np.unique(y, return_counts=True)

# Sonuçları ekrana yazdıralım
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

HU06: 3178 adet
HU07: 3728 adet
HU11: 439 adet
HU12: 676 adet
HU14: 10754 adet
HU15: 415 adet
HU19: 759 adet
UA: 130000 adet


In [76]:
# SMOTE'u kullanarak synthetic data generation yapalım
smote = SMOTE(sampling_strategy={'HU06':5822 , 'HU07':6272, 'HU11': 3541, 'HU12': 3824, 'HU14': 10754, 'HU15': 3585, 'HU19': 4500, 'UA': 130000},k_neighbors=3, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
synthetic_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['LABEL'])], axis=1)

In [77]:
synthetic_data

,PP_CINSIYET,PP_YAS,PP_MUSTERI_SEGMENTI,IL,SORU_MEDENI_HAL_RG,SORU_EGITIM_RG,SORU_GELIR_RG,SORU_COCUK_SAYISI_CVP,SORU_COCUK_SAYISI_RG,BES_AYRILMA_TALEP_ADET,...,"(50000, 100000]","(100000, 500000]","(500000, 1000000]","(1000000, 10000000]","(10000000, 1000000000000]",0,1,2-3,4+,LABEL
0,1,437,106,48,99.0,104.0,99.0,0.000000,99.0,0,...,0,0,0,0,0,1,0,0,0,UA
1,2,589,104,34,182.0,-56.0,182.0,0.000000,182.0,0,...,0,0,0,0,0,1,0,0,0,UA
2,2,441,105,6,23.0,25.0,23.0,-0.000000,-122.0,0,...,0,0,0,0,0,1,0,0,0,UA
3,2,651,104,36,117.0,-62.0,117.0,0.000000,117.0,0,...,0,0,0,0,0,1,0,0,0,UA
4,2,85,106,34,25.0,27.0,25.0,-0.000000,-122.0,0,...,0,0,0,0,0,1,0,0,0,UA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168293,1,418,103,17,8.622518,10.966888,8.622518,0.000000,-121.655629,0,...,0,0,0,0,0,0,0,0,0,HU19
168294,1,398,102,13,67.342911,70.342911,67.342911,0.239259,-52.093448,0,...,0,0,0,0,0,1,0,0,0,HU19
168295,1,392,103,39,61.953781,66.551141,61.953781,0.000000,-54.600712,0,...,0,0,0,0,0,1,0,0,0,HU19
168296,1,542,105,49,112.718124,-24.402547,112.718124,0.000000,112.718124,0,...,0,0,0,0,0,1,0,0,0,HU19


In [78]:
# Veri setini ayırma
X = synthetic_data.drop(columns = ['LABEL'], axis=1)  # Bağımsız değişkenler
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(synthetic_data['LABEL'])  # Bağımlı değişken
# y = synthetic_data['LABEL'] # Bağımlı değişken
# test_df = test_df[X.columns]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# RandomForestClassifier modelini tanımlama ve eğitme
# class_weights = {'HU06': 5, 'HU07':51, 'HU11': 10, 'HU12': 10, 'HU14': 1, 'HU15': 10, 'HU19': 10, 'UA': 0.00001}

In [48]:
unique_values, counts = np.unique(y, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

0: 5822 adet
1: 6272 adet
2: 3541 adet
3: 3824 adet
4: 10754 adet
5: 3585 adet
6: 4500 adet
7: 110000 adet


In [49]:
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

In [50]:
def custom_f1_score(y_true, y_pred):
    # F1 skorunu hesapla
    f1 = f1_score(y_true, y_pred, average=None)
    
    # Sınıf katsayıları
    class_coefficients = {
        'HU06': 0.0385,
        'HU07': 0.0328,
        'HU11': 0.2791,
        'HU12': 0.1812,
        'HU14': 0.0113,
        'HU15': 0.2952,
        'HU19': 0.1614,
        'UA': 0.0001
    }
    
    # Sınıf bazında F1 skorlarını katsayılarla çarp
    weighted_f1 = [f1[i] * class_coefficients[class_] for i, class_ in enumerate(class_coefficients.keys())]
    
    # Toplam ağırlıklı F1 skorunu döndür
    return sum(weighted_f1)

In [51]:
xgb_model = XGBClassifier()
cat_model = CatBoostClassifier()
param_grid = {
    'max_depth': [5,7,9,],
    'learning_rate': [0.1, 0.2,0.3],
}

custom_scorer = make_scorer(custom_f1_score)

# GridSearchCV'yi tanımlayın ve özel skorlama fonksiyonunu kullanın
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=custom_scorer, cv=3)

# GridSearchCV'yi uygulayın
grid_search.fit(X, y)

In [52]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

In [72]:
model = CatBoostClassifier(learning_rate=0.25, max_depth=7,iterations=1000)
model.fit(X_egitim, y_egitim)

# Test seti üzerinde tahmin yapma
# test_df = scaler.fit_transform(test_df)
tahminler = model.predict(X_test)

KeyboardInterrupt: 

In [ ]:
print(custom_f1_score(tahminler,y_test))

In [80]:
model = CatBoostClassifier(learning_rate=0.25, max_depth=7,n_estimators =1000)
model.fit(X, y,verbose=True)

# Test seti üzerinde tahmin yapma
test_df = scaler.fit_transform(test_df)
tahminler = model.predict(test_df)

0:	learn: 1.1740419	total: 709ms	remaining: 11m 48s
1:	learn: 0.9788062	total: 1.43s	remaining: 11m 55s
2:	learn: 0.8720149	total: 2.15s	remaining: 11m 56s
3:	learn: 0.7962689	total: 2.95s	remaining: 12m 14s
4:	learn: 0.7442719	total: 3.75s	remaining: 12m 26s
5:	learn: 0.6990370	total: 4.62s	remaining: 12m 45s
6:	learn: 0.6652400	total: 5.55s	remaining: 13m 6s
7:	learn: 0.6455740	total: 6.37s	remaining: 13m 9s
8:	learn: 0.6245653	total: 7.27s	remaining: 13m 20s
9:	learn: 0.6106118	total: 8.13s	remaining: 13m 24s
10:	learn: 0.5930422	total: 9.15s	remaining: 13m 42s
11:	learn: 0.5815552	total: 10.1s	remaining: 13m 47s
12:	learn: 0.5673083	total: 10.9s	remaining: 13m 44s
13:	learn: 0.5571546	total: 11.7s	remaining: 13m 43s
14:	learn: 0.5439629	total: 12.5s	remaining: 13m 41s
15:	learn: 0.5330563	total: 13.4s	remaining: 13m 45s
16:	learn: 0.5275168	total: 14.2s	remaining: 13m 42s
17:	learn: 0.5223585	total: 15.1s	remaining: 13m 42s
18:	learn: 0.5070052	total: 16s	remaining: 13m 45s
19:	lea

In [69]:
kategori_numerik_dict = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Sözlüğü ekrana yazdırma
print("Kategori -> Numerik Değer:")
for kategori, numerik_değer in kategori_numerik_dict.items():
    print(f"{kategori} -> {numerik_değer}")

Kategori -> Numerik Değer:
HU06 -> 0
HU07 -> 1
HU11 -> 2
HU12 -> 3
HU14 -> 4
HU15 -> 5
HU19 -> 6
UA -> 7


In [70]:
unique_values, counts = np.unique(y, return_counts=True)

# Sonuçları ekrana yazdıralım
for value, count in zip(unique_values, counts):
    print(f"{value}: {count} adet")

0: 5822 adet
1: 6272 adet
2: 3541 adet
3: 3824 adet
4: 10754 adet
5: 3585 adet
6: 4500 adet
7: 110000 adet


In [81]:
result_df = pd.read_csv(path+"sample_submission.csv")
result_df["LABEL"] = tahminler

label_dict = {
    0: "HU06",
    1: "HU07",
    2: "HU11",
    3: "HU12",
    4: "HU14",
    5: "HU15",
    6: "HU19",
    7: "UA"
}

# "LABEL" sütununu değiştirme
result_df["LABEL"] = result_df["LABEL"].map(label_dict)

result_df.to_csv('C:\\Users\\ACER\\Desktop\\submission.csv', index=False)

NOTLAR

TEST_DF i düzenle gelir kısmını float yap.
Predictionu gerçekleştir.
Outlier detection yapabilirsin.
